# Data reduction for the hap-fr channel

In [20]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

import astropy.units as u
from astropy.coordinates import SkyCoord,Angle
from astropy.convolution import Tophat2DKernel
from regions import CircleSkyRegion, RectangleSkyRegion

from astropy.time import Time

from pathlib import Path

In [21]:
#from gammapy.detect import compute_lima_on_off_image

from gammapy.data import DataStore
from gammapy.irf import PSFKernel
from gammapy.maps import Map, MapAxis, WcsGeom
from gammapy.datasets import MapDataset, Datasets
from gammapy.makers import (
    MapDatasetMaker,
    SafeMaskMaker,
    FoVBackgroundMaker,
)


from gammapy.modeling.models import (
    SkyModel,
    BackgroundModel,
    PowerLawSpectralModel,
    PowerLaw2SpectralModel,
    PointSpatialModel,
    ExpCutoffPowerLawSpectralModel,
    TemplateSpatialModel
)
from gammapy.modeling import Fit


import gammapy
gammapy.__version__

'0.18.2'

In [22]:
src_pos = SkyCoord(359.94, -0.04, unit="deg", frame="galactic")

In [23]:
path = Path("../../../hess_results/GC_variability_0.18.2/hap-fr")
path.mkdir(exist_ok=True)

pathm = Path(path/"mapdatasets")
pathm.mkdir(exist_ok=True)

patho = Path(path/"offsets")
patho.mkdir(exist_ok=True)

pathz = Path(path/"angzen")
pathz.mkdir(exist_ok=True)

In [24]:
data_store = DataStore.from_dir("$GAMMAPY_DATA/ash_stereo_Prod17_Calib0834_thsq64")
data_store.info()

Data store:
HDU index table:
BASE_DIR: /home/samuel/code/gammapy_data/ash_stereo_Prod17_Calib0834_thsq64
Rows: 127056
OBS_ID: 18092 -- 154591
HDU_TYPE: ['aeff', 'bkg', 'edisp', 'events', 'gti', 'psf']
HDU_CLASS: ['aeff_2d', 'bkg_2d', 'edisp_2d', 'events', 'gti', 'psf_table']


Observation table:
Observatory name: 'N/A'
Number of observations: 21186



## Data selection

In [25]:
# Sky region selection : observation pointing within this region will be selected 
#(this can be redundant with maximum offset selection, but using the max_offset should be prefered)

from astropy.coordinates import Angle

selection = dict(type='sky_circle', frame='galactic',
                 lon=Angle(0, 'deg'),
                 lat=Angle(0, 'deg'),
                 radius=Angle(1.8, 'deg'),
                 border=Angle(0, 'deg'))


# selecting the admitted range of zenithal angle, such selection can be done for any column of the obs_table
selectionZEN = dict(type='par_box', variable='ZEN_PNT', value_range=Angle([0.0,30.], 'deg'))

selectionHESS2 = dict(type='par_box', variable='OBS_ID', value_range=[85000,123635])

obs_table = data_store.obs_table.select_observations(selection)
obs_table = obs_table.select_observations(selectionZEN)

obs_table = obs_table.select_observations(selectionHESS2)

In [26]:
# Removing observations lacking IRFs (effective area, energy dispersion, point spread function and background model)

ids = obs_table["OBS_ID"].tolist()

#weird, irf missing for these ones, but it can't detect it normally somehow
#ids.remove(31539)
#ids.remove(31577)
#ids.remove(31578)
#ids.remove(31579)
#ids.remove(31580)

observations = data_store.get_observations(ids, skip_missing=True)

for obs in observations:
    try:
        obs.aeff
        obs.edisp
        obs.psf
        obs.bkg   
    except:
        ids.remove(obs.obs_id)
        print("Removed observation : " + str(obs.obs_id))
        
observations = data_store.get_observations(ids, skip_missing=True)
obs_table = obs_table.select_obs_id(ids)

## Observations en fonction du temps

In [ ]:
import matplotlib.dates as dates

data=obs_table["DATE_OBS"].tolist()
mpl_data = dates.datestr2num(data)

bins = int((mpl_data[-1] - mpl_data[0])/30)

fig, ax = plt.subplots(1,1,figsize=(20,5)) 
ax.hist(mpl_data ,bins=bins)

ax.xaxis.set_major_locator(dates.YearLocator())
ax.xaxis.set_major_formatter(dates.DateFormatter('%y'))

  
plt.title('Observations HESS')
plt.grid()
plt.xlabel("Date")
plt.ylabel("Number of observations")
    
name = "date-obs_hesstot_fr.pdf"
plt.savefig(path/name, overwrite=True)

### Time intervals

In [27]:
#time intervals used for selection

t2004  = dict(type='time_box', time_range= Time(['2004-01-01T00:00:00', '2004-12-31T23:59:59']))
t2005  = dict(type='time_box', time_range= Time(['2005-01-01T00:00:00', '2005-12-31T23:59:59']))
t2006  = dict(type='time_box', time_range= Time(['2006-01-01T00:00:00', '2006-12-31T23:59:59']))
t2007  = dict(type='time_box', time_range= Time(['2007-01-01T00:00:00', '2007-12-31T23:59:59']))
t2008  = dict(type='time_box', time_range= Time(['2008-01-01T00:00:00', '2008-12-31T23:59:59']))
t2009  = dict(type='time_box', time_range= Time(['2009-01-01T00:00:00', '2009-12-31T23:59:59']))
t2010  = dict(type='time_box', time_range= Time(['2010-01-01T00:00:00', '2010-12-31T23:59:59']))
t2011  = dict(type='time_box', time_range= Time(['2011-01-01T00:00:00', '2011-12-31T23:59:59']))
t2012  = dict(type='time_box', time_range= Time(['2012-01-01T00:00:00', '2012-12-31T23:59:59']))
t2013  = dict(type='time_box', time_range= Time(['2013-01-01T00:00:00', '2013-12-31T23:59:59']))
t2014  = dict(type='time_box', time_range= Time(['2014-01-01T00:00:00', '2014-12-31T23:59:59']))
t2015  = dict(type='time_box', time_range= Time(['2015-01-01T00:00:00', '2015-12-31T23:59:59']))
t2016  = dict(type='time_box', time_range= Time(['2016-01-01T00:00:00', '2016-12-31T23:59:59']))
t2017  = dict(type='time_box', time_range= Time(['2017-01-01T00:00:00', '2017-12-31T23:59:59']))
t2018  = dict(type='time_box', time_range= Time(['2018-01-01T00:00:00', '2018-12-31T23:59:59']))
t2019  = dict(type='time_box', time_range= Time(['2019-01-01T00:00:00', '2019-12-31T23:59:59']))

year_intervals = { 2004 : t2004, 2005 : t2005, 2006 : t2006, 2007 : t2007,
                      2008 : t2008, 2009 : t2009, 2010 : t2010, 2011 : t2011,
                      2012 : t2012, 2013 : t2013, 2014 : t2014, 2015 : t2015,
                      2016 : t2016, 2017 : t2017, 2018 : t2018, 2019 : t2019}

## Creating the geometry for the dataset

In [28]:
emin, emax = [0.5, 100] * u.TeV

energy_axis = MapAxis.from_energy_bounds(
    emin.value, emax.value, 20, unit="TeV"
)
geom = WcsGeom.create(
    skydir=(0, 0),
    binsz=0.02,
    width=(10, 8),
    frame="galactic",
    proj="CAR",
    axes=[energy_axis],
)

energy_axis_true = MapAxis.from_energy_bounds(0.3, 200, 30, unit="TeV", name="energy_true")

In [29]:
# Creating the MapDataset objects

stacked = MapDataset.create(geom=geom, energy_axis_true=energy_axis_true, name="stacked")


stacked_period = Datasets([MapDataset.create(geom=geom, energy_axis_true=energy_axis_true, name="hess1"),
                           MapDataset.create(geom=geom, energy_axis_true=energy_axis_true, name="hess1u"),
                           MapDataset.create(geom=geom, energy_axis_true=energy_axis_true, name="hess2")])

datasets_year = Datasets()

for k in range (2004,2020):
    name = f"map_{k}"
    datasets_year.append(MapDataset.create(geom=geom, energy_axis_true=energy_axis_true, name=name))

## Datasets for each year (and the complete one)

In [30]:
exclusion_region = RectangleSkyRegion(src_pos, 3*u.deg, 1*u.deg) # J1745-303 ?
exclusion_mask = geom.region_mask([exclusion_region], inside=False)
exclusion_mask = Map.from_geom(geom, data=exclusion_mask)

In [31]:
offset_max = 1.8 * u.deg
maker = MapDatasetMaker()
maker_safe_mask = SafeMaskMaker(methods=["offset-max", "bkg-peak"], offset_max=offset_max)
maker_bkg = FoVBackgroundMaker("scale", exclusion_mask)

In [ ]:
#histo_thresh = dict()
#for k in range(2004,2020): histo_thresh[k] = []

#histo_thresh_tot = []
#histo_thresh1 = []
#histo_thresh2 = []
#histo_thresh1u = []

In [ ]:
%%time

# problems starting from 31539

for obs in observations:
    #print(f"Processing obs ID: {obs.obs_id}")
    cutout = stacked.cutout(obs.pointing_radec, width=2.0 * offset_max)

    # A MapDataset is filled in this cutout geometry
    dataset = maker.run(cutout, obs)
        
    # The data quality cut is applied
    dataset = maker_safe_mask.run(dataset, obs)
        
    dataset = maker_bkg.run(dataset)
       
    year = dataset.gti.time_start.jyear.astype('int')[0]
    norm_bkg = dataset.background_model.spectral_model.norm.value
    
    if np.abs(norm_bkg - 1.) <= 0.3:
        
        datasets_year[f"map_{year}"].stack(dataset)
        stacked.stack(dataset)
        
        # Attempting to save the energy threshold (not working)
        #i = 0
        #threshold = np.max(dataset.mask_safe.data[i, :,:])
        
        #while threshold == False:
        #   i+=1
        #    threshold = np.max(dataset.mask_safe.data[0, :,:])
            
        #histo_thresh[year].append(energy_axis.edges[i].value)
        #histo_thresh_tot.append(energy_axis.edges[i].value)
        
        if (obs.obs_id > 20190 and obs.obs_id < 85000) or (obs.obs_id > 85294 and obs.obs_id < 85393):
            stacked_period["hess1"].stack(dataset)
            #histo_thresh1.append(energy_axis.edges[i].value)
                
        if (obs.obs_id > 85392 and obs.obs_id < 123635) or (obs.obs_id > 84999 and obs.obs_id < 85293):
            stacked_period["hess2"].stack(dataset)
            #histo_thresh2.append(energy_axis.edges[i].value)
                
        if (obs.obs_id > 129418 and obs.obs_id < 153843):
            stacked_period["hess1u"].stack(dataset)
            #histo_thresh1u.append(energy_axis.edges[i].value)
    #else:
    #    print(f"Bkg norm is out of bounds: {norm_bkg:.2f}. Rejecting observation {obs.obs_id}.")

In [32]:
%%time

#finding the issue with HESS2

for obs in observations:
    #print(f"Processing obs ID: {obs.obs_id}")
    cutout = stacked.cutout(obs.pointing_radec, width=2.0 * offset_max)

    # A MapDataset is filled in this cutout geometry
    dataset = maker.run(cutout, obs)
        
    # The data quality cut is applied
    dataset = maker_safe_mask.run(dataset, obs)
        
    dataset = maker_bkg.run(dataset)
       
    year = dataset.gti.time_start.jyear.astype('int')[0]
    norm_bkg = dataset.background_model.spectral_model.norm.value
    
    if np.abs(norm_bkg - 1.) <= 0.3:
        
        datasets_year[f"map_{year}"].stack(dataset)
        
        # Attempting to save the energy threshold (not working)
        #i = 0
        #threshold = np.max(dataset.mask_safe.data[i, :,:])
        
        #while threshold == False:
        #   i+=1
        #    threshold = np.max(dataset.mask_safe.data[0, :,:])
            
        #histo_thresh[year].append(energy_axis.edges[i].value)
        #histo_thresh_tot.append(energy_axis.edges[i].value)
     
        if (obs.obs_id > 85392 and obs.obs_id < 123635) or (obs.obs_id > 84999 and obs.obs_id < 85293):  # mai-juillet 2013 jusqu'à fin 2016
            stacked_period["hess2"].stack(dataset)
            #histo_thresh2.append(energy_axis.edges[i].value)


/home/samuel/anaconda3/envs/gammapy-0.18.2/lib/python3.7/site-packages/astropy/units/quantity.py:477: RuntimeWarning: invalid value encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/home/samuel/anaconda3/envs/gammapy-0.18.2/lib/python3.7/site-packages/astropy/units/quantity.py:477: RuntimeWarning: invalid value encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/home/samuel/anaconda3/envs/gammapy-0.18.2/lib/python3.7/site-packages/astropy/units/quantity.py:477: RuntimeWarning: invalid value encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/home/samuel/anaconda3/envs/gammapy-0.18.2/lib/python3.7/site-packages/astropy/units/quantity.py:477: RuntimeWarning: invalid value encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/home/samuel/anaconda3/envs/gammapy-0.18.2/lib/python3.7/site-packages/astro

CPU times: user 6min 7s, sys: 1.29 s, total: 6min 8s
Wall time: 6min 9s


In [33]:
print(datasets_year["map_2016"])

MapDataset
----------

  Name                            : map_2016 

  Total counts                    : 5312 
  Total background counts         : 4957.68
  Total excess counts             : 354.32

  Predicted counts                : 4957.68
  Predicted background counts     : 4957.68
  Predicted excess counts         : nan

  Exposure min                    : 2.30e+08 m2 s
  Exposure max                    : 4.08e+09 m2 s

  Number of total bins            : 4000000 
  Number of fit bins              : 684988 

  Fit statistic type              : cash
  Fit statistic value (-2 log(L)) : nan

  Number of models                : 0 
  Number of parameters            : 0
  Number of free parameters       : 0




In [28]:
filename = "HESS2/mapdataset_hess2_30-50angzen.fits.gz"
stacked_period["hess2"].write(pathm/filename, overwrite=True)

In [34]:
filename = "HESS2/mapdataset_hess2_2013_30-50angzen.fits.gz"
datasets_year["map_2013"].write(pathm/filename, overwrite=True)

filename = "HESS2/mapdataset_hess2_2014_30-50angzen.fits.gz"
datasets_year["map_2014"].write(pathm/filename, overwrite=True)

filename = "HESS2/mapdataset_hess2_2015_30-50angzen.fits.gz"
datasets_year["map_2015"].write(pathm/filename, overwrite=True)

filename = "HESS2/mapdataset_hess2_2016_30-50angzen.fits.gz"
datasets_year["map_2016"].write(pathm/filename, overwrite=True)

## Saving the datasets

In [ ]:
filename = "mapdataset_tot.fits.gz"
stacked.write(pathm/filename, overwrite=True)

stacked_period.write(pathm/"datasets_period.yaml", overwrite=True)

datasets_year.write(pathm/"datasets_year.yaml", overwrite=True)

# Collecting data on observation conditions

## Offset angles

In [ ]:
# Extracing offsets of observations (relative to SgrA*) for each year

sgra_pos = SkyCoord(359.94, -0.04, unit="deg", frame="galactic")

histo_offset = dict()
offset_tot = []
offset_hess1 = []
offset_hess2 = []
offset_hess1u = []

for year in range(2004,2020):
    histo_offset[year] = []

for obs in observations:
    year = obs.gti.time_start.jyear.astype('int')[0]
    
    direction = obs.pointing_radec
    sep = direction.separation(sgra_pos) #calcule la séparation "on sky" entre les deux coordonnées
    
    histo_offset[year].append(sep.value)
    offset_tot.append(sep.value)
        
        
for year in range(2004,2020):        
    plt.figure()
    plt.hist(histo_offset[year], 40, (0.0, 2.1))
    
    plt.title(str(year))
    plt.grid()
    plt.xlabel("Offset with respect to the central source (deg)")
    plt.ylabel("Number of observations")
    
    name = "offsets_"+str(year)+"_hd.pdf"
    plt.savefig(patho/name, overwrite=True)
    

plt.figure()
plt.hist(offset_tot, 40, (0.0, 2.1))
    
plt.title("Total (2004-2019)")
plt.grid()
plt.xlabel("Offset with respect to the central source(deg)")
plt.ylabel("Number of observations")
name = 'offset_tot_hd.pdf'
plt.savefig(patho/name, overwrite=True)

In [ ]:
plt.figure()
plt.hist(offset_hess1, 40, (0.0, 2.1))
    
plt.title("Total HESS 1")
plt.grid()
plt.xlabel("Offset with respect to the central source(deg)")
plt.ylabel("Number of observations")
name = 'offset_hess1_hd.pdf'
plt.savefig(patho/name, overwrite=True)


plt.figure()
plt.hist(offset_hess1u, 40, (0.0, 2.1))
    
plt.title("Total HESS 1U")
plt.grid()
plt.xlabel("Offset with respect to the central source(deg)")
plt.ylabel("Number of observations")
name = 'offset_hess1u_hd.pdf'
plt.savefig(patho/name, overwrite=True)


plt.figure()
plt.hist(offset_hess2, 40, (0.0, 2.1))
    
plt.title("Total HESS 2")
plt.grid()
plt.xlabel("Offset with respect to the central source(deg)")
plt.ylabel("Number of observations")
name = 'offset_hess2_hd.pdf'
plt.savefig(patho/name, overwrite=True)

## Zenith angles

In [ ]:
# Extracing zenithal angle of observation for each year

histo_zen = dict()

pathz = Path(path/"angzen")
pathz.mkdir(exist_ok=True)

for year in range(2004,2020):  
    histo_zen[year] = []
    
    obs_table_year = obs_table.select_observations(year_intervals[year])
    histo_zen[year] = obs_table_year["ZEN_PNT"].tolist()
    
    plt.figure()
    plt.hist(histo_zen[year], 40, (5.0, 65.0))
    
    plt.title(str(year))
    plt.grid()
    plt.xlabel("Zenithal angle (deg)")
    plt.ylabel("Number of observations")
    
    name = "angzen_"+str(year)+"fr.pdf"
    plt.savefig(pathz/name, overwrite=True)

In [ ]:
zen_tot = obs_table["ZEN_PNT"].tolist()
avg = np.mean(zen_tot)

plt.figure()
plt.hist(zen_tot, 40, (5.0, 65.0))
    
plt.title('Total (2004-2019), mean = {0:0.2f}°'.format(avg))
plt.grid()
plt.xlabel("Zenith angle (deg)")
plt.ylabel("Number of observations")

 
name = "angzen_tot_fr.pdf"
plt.savefig(pathz/name, overwrite=True)

## Adding up LiveTime for each year

In [ ]:
res = []

for year in range(2004,2020):
    obs_table_year = obs_table.select_observations(year_intervals[year])
    res.append(sum(obs_table_year["LIVETIME"].tolist())/3600)


plt.figure(figsize=(10,8))

plt.plot(range(2004,2020), res)
plt.grid()
plt.xlabel("Year")
plt.ylabel("Total LIVETIME (hours)")
    
name = path/"livetime_plot_fr.pdf"
plt.savefig(name, overwrite=True)

## Energy thresholds